# Corpus Merging.

## CSAT 만들기

CSAT의 경우, 자체적인 POS 값들 달기 애매하기 때문에 E-lexicon proj에서 제공하는 POS 값을 우선적으로 가져와서 붙여 넣자. 아마 대부분 있지 않을까?

### Module Import

In [2]:
import pandas as pd # 데이터 처리
import numpy as np # 데이터 처리
import seaborn as sns # 데이터 시각화
from tqdm import tqdm # 데이터 처리
import re # 정규식으로 특수기호 및 char 처리
import json # 데이터 가공 후 저장할 형식
from typing import Any, Dict, List  # python data-type 정의
from pathlib import Path # 파일 경로 처리
import os # 파일 경로 및 처리
from json_handler import JsonFileHandler
from preprocessor import Preprocessor, TargetCorpusPreprocessor

### file path setting

In [10]:
# 파일 경로 확인.
proj_path = os.path.dirname(os.path.dirname(Path.cwd()))
corpora_dir = os.path.join(proj_path, 'Corpora')
CSAT_json_path = save_path = os.path.join(corpora_dir, 'CSAT-latest')

# 파일 경로 유효성 확인.
if not os.path.exists(proj_path):
    raise FileNotFoundError(f"Project path not found at {proj_path}")
if not os.path.exists(CSAT_json_path):
    raise FileNotFoundError(f"CSAT json path not found at {CSAT_json_path}")
if not any(file.endswith('.json') for file in os.listdir(CSAT_json_path)):
    raise FileExistsError(f"Any .json file doesn't exist in {CSAT_json_path}")
print("All file paths are valid.")

All file paths are valid.


### Utils: generate_word_list
중복을 허용하여 단어를 extend 한 list 생성.
이후 pandas 다른 method(ex: value_count)를 사용해서 빈도수 계산.

In [14]:
def generate_word_list(document_list: list, document_dir: str):
    # article 별 단어 분석.
    word_list = []   # word 정보를 담을 리스트
    json_handler = JsonFileHandler()

    for document_name in document_list: # corpus에서 하나의 json 파일을 가져와서
        document = json_handler.load_data(os.path.join(document_dir, document_name))  # 하나의 json 파일 내에 있는 모든 파일을 가져옴.

        if isinstance(document, list):   # 한 json 내에 여러 obj가 있는 경우, 
            for sentence in document:  # json 파일 내 있는 여러 문장들 수 
                word_list.extend(sentence['tokens'])
        else:   # 한 json 내에 하나의 obj가 있는 경우, 
            word_list.extend(document['tokens'])   

    return word_list

### CSAT(only): 수능 + 모의고사

In [ ]:
json_handler = JsonFileHandler()
CSAT_path = os.path.join(corpora_dir, 'CSAT-latest')
corpus_list = os.listdir(CSAT_path)
corpus_list = [file for file in corpus_list if file.endswith('.json')]
reading_corpus = [file for file in corpus_list if 'reading' in file]
listening_corpus = [file for file in corpus_list if 'listening' in file]

len(reading_corpus), len(listening_corpus)

(52, 49)

In [40]:
listening_word_list = generate_word_list(listening_corpus, CSAT_path)
reading_word_list = generate_word_list(reading_corpus, CSAT_path)
CSAT_only_word_list = pd.Series(listening_word_list + reading_word_list)

In [41]:
CSAT_only_word_list.shape

(58328,)

In [42]:
CSAT_only_word_list.value_counts()

.              3435
,              2787
the            2748
to             1627
of             1376
               ... 
substantial       1
tolls             1
merchants         1
immediacy         1
doubly            1
Name: count, Length: 7282, dtype: int64

In [45]:
CSAT_only_word_list_df = CSAT_only_word_list.value_counts().to_frame().reset_index().rename(columns={'index': 'Word', 'count': 'Freq'})
CSAT_only_word_list_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_only.xlsx'), index=False)
CSAT_only_word_list_df

,Word,Freq
0,.,3435
1,",",2787
2,the,2748
3,to,1627
4,of,1376
...,...,...
7277,substantial,1
7278,tolls,1
7279,merchants,1
7280,immediacy,1


### textbook(only)

In [12]:
json_handler = JsonFileHandler()
CSAT_path = os.path.join(corpora_dir, 'CSAT-latest')
corpus_list = os.listdir(CSAT_path)
corpus_list = [file for file in corpus_list if file.endswith('.json')]
textbook_corpus = [file for file in corpus_list if 'textbook' in file]

len(textbook_corpus)

71

In [46]:
textbook_word_list = generate_word_list(textbook_corpus, CSAT_path)
textbook_only_word_list = pd.Series(textbook_word_list)

In [47]:
textbook_only_word_list.shape

(9043,)

In [48]:
textbook_only_word_list.value_counts()

.            530
the          401
,            398
to           254
and          240
            ... 
finale         1
shoulders      1
teammates      1
field          1
enjoyed        1
Name: count, Length: 2076, dtype: int64

In [49]:
textbook_only_word_list_df = textbook_only_word_list.value_counts().to_frame().reset_index().rename(columns={'index': 'Word', 'count': 'Freq'})
textbook_only_word_list_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'textbook_only.xlsx'), index=False)
textbook_only_word_list_df

,Word,Freq
0,.,530
1,the,401
2,",",398
3,to,254
4,and,240
...,...,...
2071,finale,1
2072,shoulders,1
2073,teammates,1
2074,field,1


### CSAT Total

In [13]:
json_handler = JsonFileHandler()
CSAT_path = os.path.join(corpora_dir, 'CSAT-latest')
corpus_list = os.listdir(CSAT_path)
corpus_list = [file for file in corpus_list if file.endswith('.json')]
reading_corpus = [file for file in corpus_list if 'reading' in file]
listening_corpus = [file for file in corpus_list if 'listening' in file]
textbook_corpus = [file for file in corpus_list if 'textbook' in file]

len(reading_corpus), len(listening_corpus), len(textbook_corpus)

(52, 49, 71)

In [ ]:
listening_word_list = generate_word_list(listening_corpus, CSAT_path)
reading_word_list = generate_word_list(reading_corpus, CSAT_path)
textbook_word_list = generate_word_list(textbook_corpus, CSAT_path)
CSAT_word_list = pd.Series(listening_word_list + reading_word_list + textbook_word_list)

In [23]:
CSAT_word_list.shape

(67371,)

In [26]:
CSAT_word_list.value_counts()

.               3965
,               3185
the             3149
to              1881
of              1561
                ... 
reconnects         1
trolly             1
conveniently       1
tagging            1
treasured          1
Name: count, Length: 7796, dtype: int64

In [50]:
CSAT_word_list_df = CSAT_word_list.value_counts().to_frame().reset_index().rename(columns={'index': 'Word', 'count': 'Freq'})
CSAT_word_list_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT.xlsx'), index=False)
CSAT_word_list_df

,Word,Freq
0,.,3965
1,",",3185
2,the,3149
3,to,1881
4,of,1561
...,...,...
7791,reconnects,1
7792,trolly,1
7793,conveniently,1
7794,tagging,1


## E-lexicon proj

E-lexicon proj를 보면, a의 orthographic N이 1이다. 즉, b,c,d,e 같이 치환된 케이스 중 비단어는 제외했다는 말이니, CSAT도 그대로 사용하기는 무리. 따라서 E-lexicon이랑 intersection을 먼저 시켜봐야 할 듯.

In [32]:
E_lexicon_proj_path = os.path.join(corpora_dir, 'E-lexicon-proj')

In [34]:
E_lexicon_df = pd.read_csv(os.path.join(E_lexicon_proj_path, 'English Lexicon Project Items original.csv'))
E_lexicon_df.head()

,Word,Length,Freq_HAL,Log_Freq_HAL,SUBTLWF,LgSUBTLWF,SUBTLCD,LgSUBTLCD,Ortho_N,OLD,POS
0,a,1,"10,610,626",16.177,"20,415.270",6.018,99.930,3.923,1,1.450,minor|NN
1,aah,3,222,5.403,52.710,3.430,7.560,2.803,2,1.850,#
2,Aaron,5,"10,806",9.288,14.650,2.874,1.930,2.212,3,1.850,NN
3,aback,5,387,5.958,0.290,1.204,0.180,1.204,0,1.950,RB
4,abacus,6,513,6.240,0.240,1.114,0.120,1.041,0,2.900,NN


In [120]:
# null이라는 단어가 있는데, type 지정을 안 해주면 null 값으로 분류됨.
E_lexicon_df['Word'] = E_lexicon_df['Word'].astype(str)
E_lexicon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40481 entries, 0 to 40480
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Word          40481 non-null  object 
 1   Length        40481 non-null  int64  
 2   Freq_HAL      40481 non-null  object 
 3   Log_Freq_HAL  40481 non-null  float64
 4   SUBTLWF       40481 non-null  object 
 5   LgSUBTLWF     40481 non-null  object 
 6   SUBTLCD       40481 non-null  object 
 7   LgSUBTLCD     40481 non-null  object 
 8   Ortho_N       40481 non-null  int64  
 9   OLD           40481 non-null  object 
 10  POS           40481 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 3.4+ MB


E-lexicon proj에서도 Word는 lower 적용.

In [121]:
E_lexicon_df['Word'] = E_lexicon_df['Word'].str.lower()
E_lexicon_df.head()

,Word,Length,Freq_HAL,Log_Freq_HAL,SUBTLWF,LgSUBTLWF,SUBTLCD,LgSUBTLCD,Ortho_N,OLD,POS
0,a,1,"10,610,626",16.177,"20,415.270",6.018,99.930,3.923,1,1.450,minor|NN
1,aah,3,222,5.403,52.710,3.430,7.560,2.803,2,1.850,#
2,aaron,5,"10,806",9.288,14.650,2.874,1.930,2.212,3,1.850,NN
3,aback,5,387,5.958,0.290,1.204,0.180,1.204,0,1.950,RB
4,abacus,6,513,6.240,0.240,1.114,0.120,1.041,0,2.900,NN


In [122]:
E_lexicon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40481 entries, 0 to 40480
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Word          40481 non-null  object 
 1   Length        40481 non-null  int64  
 2   Freq_HAL      40481 non-null  object 
 3   Log_Freq_HAL  40481 non-null  float64
 4   SUBTLWF       40481 non-null  object 
 5   LgSUBTLWF     40481 non-null  object 
 6   SUBTLCD       40481 non-null  object 
 7   LgSUBTLCD     40481 non-null  object 
 8   Ortho_N       40481 non-null  int64  
 9   OLD           40481 non-null  object 
 10  POS           40481 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 3.4+ MB


In [145]:
from preprocessor import TargetCorpusPreprocessor

E_lexicon_df = TargetCorpusPreprocessor(E_lexicon_df).preprocess()
E_lexicon_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'E_lexicon_preprocessed.xlsx'), index=False)

Cleaning information col by col:   0%|          | 0/11 [00:00<?, ?it/s]

Freq_HAL
SUBTLWF


Cleaning information col by col:  45%|████▌     | 5/11 [00:02<00:02,  2.35it/s]

LgSUBTLWF


Cleaning information col by col:  55%|█████▍    | 6/11 [00:04<00:03,  1.27it/s]

SUBTLCD


Cleaning information col by col:  64%|██████▎   | 7/11 [00:06<00:04,  1.11s/it]

LgSUBTLCD


Cleaning information col by col: 100%|██████████| 11/11 [00:08<00:00,  1.24it/s]

OLD
POS


In [168]:
E_lexicon_df.head()

,Word,Length,Freq_HAL,Log_Freq_HAL,SUBTLWF,LgSUBTLWF,SUBTLCD,LgSUBTLCD,Ortho_N,OLD,POS
0,a,1,10610626,16.177,20415.27,6.018,99.930,3.923,1,1.45,minor|NN
1,aah,3,222,5.403,52.71,3.430,7.560,2.803,2,1.85,None
2,aaron,5,10806,9.288,14.65,2.874,1.930,2.212,3,1.85,NN
3,aback,5,387,5.958,0.29,1.204,0.180,1.204,0,1.95,RB
4,abacus,6,513,6.240,0.24,1.114,0.120,1.041,0,2.90,NN


### CSAT_edit

In [146]:
CSAT_only_df = pd.read_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_only.xlsx'))
textbook_only_df = pd.read_excel(os.path.join(corpora_dir, 'merged_corpora', 'textbook_only.xlsx'))
CSAT_df = pd.read_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT.xlsx'))

# overlap 1562개
CSAT_df.shape, CSAT_only_df.shape,textbook_only_df.shape

((7796, 2), (7282, 2), (2076, 2))

In [147]:
CSAT_only_POS_df = pd.merge(CSAT_only_df, E_lexicon_df, on='Word', how='left')[['Word', 'Freq', 'POS']]
CSAT_only_POS_df = CSAT_only_POS_df.dropna(subset=['POS']).reset_index(drop=True)
CSAT_only_POS_df

,Word,Freq,POS
0,the,2748,minor
1,to,1627,minor
2,of,1376,minor
3,and,1253,minor
4,a,1098,minor|NN
...,...,...,...
6526,substantial,1,JJ
6527,tolls,1,NN|VB
6528,merchants,1,NN
6529,immediacy,1,NN


In [148]:
textbook_only_POS_df = pd.merge(textbook_only_df, E_lexicon_df, on='Word', how='left')[['Word', 'Freq', 'POS']]
textbook_only_POS_df = textbook_only_POS_df.dropna(subset=['POS']).reset_index(drop=True)
textbook_only_POS_df

,Word,Freq,POS
0,the,401,minor
1,to,254,minor
2,and,240,minor
3,a,186,minor|NN
4,of,185,minor
...,...,...,...
1903,finale,1,NN
1904,shoulders,1,NN|VB
1905,teammates,1,NN
1906,field,1,NN|VB


각각 E-lexicon이랑 합치고 POS 달기

In [184]:
CSAT_POS_df = pd.merge(CSAT_df, E_lexicon_df, on='Word', how='left')[['Word', 'Freq', 'POS']]
CSAT_POS_df = CSAT_POS_df.dropna(subset=['POS']).reset_index(drop=True)
CSAT_POS_df

,Word,Freq,POS
0,the,3149,minor
1,to,1881,minor
2,of,1561,minor
3,and,1493,minor
4,a,1284,minor|NN
...,...,...,...
6910,palms,1,NN|VB
6911,renewing,1,VB
6912,conveniently,1,RB
6913,tagging,1,VB


add Ortho_N, OLD20

In [185]:
from neighborhood import orthographic_N, OLD20

lexicon = CSAT_POS_df['Word'].to_list() 

CSAT_POS_df['Ortho_N_CSAT'] = CSAT_POS_df['Word'].apply(lambda x: orthographic_N(x, lexicon))
CSAT_POS_df['OLD20_CSAT'] = CSAT_POS_df['Word'].apply(lambda x: OLD20(x, lexicon))
CSAT_POS_df

,Word,Freq,POS,Ortho_N_CSAT,OLD20_CSAT
0,the,3149,minor,1,1.65
1,to,1881,minor,4,1.45
2,of,1561,minor,5,1.60
3,and,1493,minor,5,1.30
4,a,1284,minor|NN,1,1.50
...,...,...,...,...,...
6910,palms,1,NN|VB,0,1.90
6911,renewing,1,VB,0,2.80
6912,conveniently,1,RB,0,4.55
6913,tagging,1,VB,0,2.50


In [187]:
lexicon = CSAT_only_POS_df['Word'].to_list() 

CSAT_only_POS_df['Ortho_N_CSAT(only)'] = CSAT_only_POS_df['Word'].apply(lambda x: orthographic_N(x, lexicon))
CSAT_only_POS_df['OLD20_CSAT(only)'] = CSAT_only_POS_df['Word'].apply(lambda x: OLD20(x, lexicon))
CSAT_only_POS_df

,Word,CSAT_only_Freq,POS,Ortho_N_CSAT(only),OLD20_CSAT(only)
0,the,2748,minor,1,1.65
1,to,1627,minor,4,1.45
2,of,1376,minor,5,1.60
3,and,1253,minor,5,1.35
4,a,1098,minor|NN,1,1.50
...,...,...,...,...,...
6526,substantial,1,JJ,0,5.00
6527,tolls,1,NN|VB,3,1.75
6528,merchants,1,NN,0,3.70
6529,immediacy,1,NN,0,4.40


In [188]:
lexicon = textbook_only_POS_df['Word'].to_list() 

textbook_only_POS_df['Ortho_N_Textbook'] = textbook_only_POS_df['Word'].apply(lambda x: orthographic_N(x, lexicon))
textbook_only_POS_df['OLD20_Textbook'] = textbook_only_POS_df['Word'].apply(lambda x: OLD20(x, lexicon))
textbook_only_POS_df

,Word,Textbook_only_Freq,POS,Ortho_N_Textbook,OLD20_Textbook
0,the,401,minor,1,1.65
1,to,254,minor,4,1.50
2,and,240,minor,3,1.65
3,a,186,minor|NN,1,1.65
4,of,185,minor,4,1.65
...,...,...,...,...,...
1903,finale,1,NN,0,2.60
1904,shoulders,1,NN|VB,0,4.20
1905,teammates,1,NN,0,4.60
1906,field,1,NN|VB,0,2.35


In [189]:
CSAT_only_POS_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_only_POS_N.xlsx'), index=False)
textbook_only_POS_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'textbook_only_POS_N.xlsx'), index=False)
CSAT_POS_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_POS_N.xlsx'), index=False)

In [192]:
CSAT_only_POS_df.columns= ['Word', 'CSAT_only_Freq', 'POS', 'Ortho_N_CSAT(only)', 'OLD20_CSAT(only)']
textbook_only_POS_df.columns= ['Word', 'Textbook_only_Freq', 'POS', 'Ortho_N_Textbook(only)', 'OLD20_Textbook(only)']
CSAT_vs_Textbook_df = pd.merge(CSAT_only_POS_df, textbook_only_POS_df, on='Word', how='inner').drop(columns=['POS_y']).rename(columns={'POS_x': 'POS'})
CSAT_vs_Textbook_df

,Word,CSAT_only_Freq,POS,Ortho_N_CSAT(only),OLD20_CSAT(only),Textbook_only_Freq,Ortho_N_Textbook(only),OLD20_Textbook(only)
0,the,2748,minor,1,1.65,401,1,1.65
1,to,1627,minor,4,1.45,254,4,1.50
2,of,1376,minor,5,1.60,185,4,1.65
3,and,1253,minor,5,1.35,240,3,1.65
4,a,1098,minor|NN,1,1.50,186,1,1.65
...,...,...,...,...,...,...,...,...
1519,memorable,1,JJ,0,3.55,1,0,4.35
1520,perspectives,1,NN,0,4.40,1,0,5.30
1521,cools,1,VB,2,1.75,1,1,1.90
1522,gesture,1,NN,0,2.55,1,0,2.80


In [196]:
CSAT_vs_Textbook_df = pd.merge(CSAT_only_POS_df, textbook_only_POS_df, on='Word', how='outer').drop(columns=['POS_y']).rename(columns={'POS_x': 'POS'})
CSAT_vs_Textbook_df

,Word,CSAT_only_Freq,POS,Ortho_N_CSAT(only),OLD20_CSAT(only),Textbook_only_Freq,Ortho_N_Textbook(only),OLD20_Textbook(only)
0,a,1098.0,minor|NN,1.0,1.50,186.0,1.0,1.65
1,abandoned,NaN,NaN,NaN,NaN,1.0,0.0,4.70
2,abilities,2.0,NN,0.0,3.40,NaN,NaN,NaN
3,ability,16.0,NN,0.0,3.25,NaN,NaN,NaN
4,able,21.0,JJ,0.0,1.85,8.0,0.0,2.45
...,...,...,...,...,...,...,...,...
6910,zebra,2.0,NN,0.0,2.80,NaN,NaN,NaN
6911,zero,1.0,minor|NN|VB,0.0,2.10,NaN,NaN,NaN
6912,zone,2.0,NN,5.0,1.50,NaN,NaN,NaN
6913,zones,1.0,NN,2.0,1.70,NaN,NaN,NaN


In [ ]:
CSAT_vs_Textbook_df = CSAT_vs_Textbook_df[['Word', 'CSAT_only_Freq', 'Textbook_only_Freq', 'Ortho_N_CSAT(only)', 'Ortho_N_Textbook(only)',  'OLD20_CSAT(only)',  'OLD20_Textbook(only)', 'POS']]
CSAT_vs_Textbook_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_vs_Textbook_POS_N_outer.xlsx'), index=False)
CSAT_vs_Textbook_df

,Word,CSAT_only_Freq,Textbook_only_Freq,Ortho_N_CSAT(only),Ortho_N_Textbook(only),OLD20_CSAT(only),OLD20_Textbook(only),POS
0,a,1098.0,186.0,1.0,1.0,1.50,1.65,minor|NN
1,abandoned,NaN,1.0,NaN,0.0,NaN,4.70,NaN
2,abilities,2.0,NaN,0.0,NaN,3.40,NaN,NN
3,ability,16.0,NaN,0.0,NaN,3.25,NaN,NN
4,able,21.0,8.0,0.0,0.0,1.85,2.45,JJ
...,...,...,...,...,...,...,...,...
6910,zebra,2.0,NaN,0.0,NaN,2.80,NaN,NN
6911,zero,1.0,NaN,0.0,NaN,2.10,NaN,minor|NN|VB
6912,zone,2.0,NaN,5.0,NaN,1.50,NaN,NN
6913,zones,1.0,NaN,2.0,NaN,1.70,NaN,NN


In [ ]:
CSAT_POS_df.columns =['Word', 'CSAT_Freq', 'POS', 'Ortho_N_CSAT', 'OLD20_CSAT']
CSAT_POS_df = CSAT_POS_df[['Word', 'CSAT_Freq', 'Ortho_N_CSAT', 'OLD20_CSAT', 'POS']]
CSAT_POS_df


,Word,CSAT_Freq,Ortho_N_CSAT,OLD20_CSAT,POS
0,the,3149,1,1.65,minor
1,to,1881,4,1.45,minor
2,of,1561,5,1.60,minor
3,and,1493,5,1.30,minor
4,a,1284,1,1.50,minor|NN
...,...,...,...,...,...
6910,palms,1,0,1.90,NN|VB
6911,renewing,1,0,2.80,VB
6912,conveniently,1,0,4.55,RB
6913,tagging,1,0,2.50,VB


In [236]:
CSAT_df = pd.merge(CSAT_POS_df.drop(columns=['POS']), CSAT_vs_Textbook_df, on='Word', how='outer')[['Word', 'CSAT_Freq', 'CSAT_only_Freq', 'Textbook_only_Freq', 'Ortho_N_CSAT', 'Ortho_N_CSAT(only)', 'Ortho_N_Textbook(only)', 'OLD20_CSAT', 'OLD20_CSAT(only)', 'OLD20_Textbook(only)', 'POS']]
CSAT_df['Length'] = CSAT_df['Word'].apply(lambda x: len(x))
relative_unit = 1000000
CSAT_df['CSAT_RFreq'] = CSAT_df['CSAT_Freq'] / relative_unit
CSAT_df['CSAT_only_RFreq'] = CSAT_df['CSAT_only_Freq'] / relative_unit
CSAT_df['Textbook_only_RFreq'] = CSAT_df['Textbook_only_Freq'] / relative_unit
CSAT_df = CSAT_df[['Word', 'Length', 'CSAT_Freq', 'CSAT_only_Freq', 'Textbook_only_Freq', 'CSAT_RFreq', 'CSAT_only_RFreq', 'Textbook_only_RFreq', 'Ortho_N_CSAT', 'Ortho_N_CSAT(only)', 'Ortho_N_Textbook(only)', 'OLD20_CSAT', 'OLD20_CSAT(only)', 'OLD20_Textbook(only)', 'POS']]
CSAT_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_final.xlsx'), index=False)
CSAT_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6915 entries, 0 to 6914
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Word                    6915 non-null   object 
 1   Length                  6915 non-null   int64  
 2   CSAT_Freq               6915 non-null   int64  
 3   CSAT_only_Freq          6531 non-null   float64
 4   Textbook_only_Freq      1908 non-null   float64
 5   CSAT_RFreq              6915 non-null   float64
 6   CSAT_only_RFreq         6531 non-null   float64
 7   Textbook_only_RFreq     1908 non-null   float64
 8   Ortho_N_CSAT            6915 non-null   int64  
 9   Ortho_N_CSAT(only)      6531 non-null   float64
 10  Ortho_N_Textbook(only)  1908 non-null   float64
 11  OLD20_CSAT              6915 non-null   float64
 12  OLD20_CSAT(only)        6531 non-null   float64
 13  OLD20_Textbook(only)    1908 non-null   float64
 14  POS                     6531 non-null   

### 어떤 단어가 HAL에 없을까?

In [237]:
CSAT_df['POS'] = CSAT_df['POS'].astype(str)
CSAT_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6915 entries, 0 to 6914
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Word                    6915 non-null   object 
 1   Length                  6915 non-null   int64  
 2   CSAT_Freq               6915 non-null   int64  
 3   CSAT_only_Freq          6531 non-null   float64
 4   Textbook_only_Freq      1908 non-null   float64
 5   CSAT_RFreq              6915 non-null   float64
 6   CSAT_only_RFreq         6531 non-null   float64
 7   Textbook_only_RFreq     1908 non-null   float64
 8   Ortho_N_CSAT            6915 non-null   int64  
 9   Ortho_N_CSAT(only)      6531 non-null   float64
 10  Ortho_N_Textbook(only)  1908 non-null   float64
 11  OLD20_CSAT              6915 non-null   float64
 12  OLD20_CSAT(only)        6531 non-null   float64
 13  OLD20_Textbook(only)    1908 non-null   float64
 14  POS                     6915 non-null   

In [240]:
CSAT_df.describe()

,Length,CSAT_Freq,CSAT_only_Freq,Textbook_only_Freq,CSAT_RFreq,CSAT_only_RFreq,Textbook_only_RFreq,Ortho_N_CSAT,Ortho_N_CSAT(only),Ortho_N_Textbook(only),OLD20_CSAT,OLD20_CSAT(only),OLD20_Textbook(only)
count,6915.000000,6915.000000,6531.000000,1908.000000,6915.000000,6531.000000,1908.000000,6915.000000,6531.000000,1908.000000,6915.000000,6531.000000,1908.000000
mean,7.199132,8.375705,7.667738,4.109015,0.000008,0.000008,0.000004,0.815907,0.779360,0.574423,2.906175,2.928885,3.064099
std,2.399643,63.256332,56.290138,16.251551,0.000063,0.000056,0.000016,1.705656,1.646987,1.174013,1.057313,1.061700,1.166500
min,1.000000,1.000000,1.000000,1.000000,0.000001,0.000001,0.000001,0.000000,0.000000,0.000000,1.000000,1.000000,1.350000
25%,5.000000,1.000000,1.000000,1.000000,0.000001,0.000001,0.000001,0.000000,0.000000,0.000000,2.000000,2.050000,2.150000
50%,7.000000,2.000000,2.000000,1.000000,0.000002,0.000002,0.000001,0.000000,0.000000,0.000000,2.700000,2.700000,2.800000
75%,9.000000,4.000000,4.000000,2.000000,0.000004,0.000004,0.000002,1.000000,1.000000,1.000000,3.550000,3.550000,3.650000
max,16.000000,3149.000000,2748.000000,401.000000,0.003149,0.002748,0.000401,14.000000,12.000000,8.000000,8.900000,8.350000,9.500000


In [238]:
CSAT_minus_HAL = CSAT_df[CSAT_df['POS'] == 'nan']
CSAT_minus_HAL.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_minus_HAL.xlsx'), index=False)
CSAT_minus_HAL

,Word,Length,CSAT_Freq,CSAT_only_Freq,Textbook_only_Freq,CSAT_RFreq,CSAT_only_RFreq,Textbook_only_RFreq,Ortho_N_CSAT,Ortho_N_CSAT(only),Ortho_N_Textbook(only),OLD20_CSAT,OLD20_CSAT(only),OLD20_Textbook(only),POS
1,abandoned,9,1,NaN,1.0,0.000001,NaN,0.000001,0,NaN,0.0,4.40,NaN,4.70,nan
9,abroad,6,1,NaN,1.0,0.000001,NaN,0.000001,0,NaN,0.0,2.60,NaN,2.75,nan
40,accordance,10,1,NaN,1.0,0.000001,NaN,0.000001,0,NaN,0.0,4.10,NaN,5.15,nan
44,accountability,14,1,NaN,1.0,0.000001,NaN,0.000001,0,NaN,0.0,6.25,NaN,7.90,nan
72,activist,8,1,NaN,1.0,0.000001,NaN,0.000001,0,NaN,0.0,3.50,NaN,4.05,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6836,wonderland,10,1,NaN,1.0,0.000001,NaN,0.000001,0,NaN,0.0,4.35,NaN,4.95,nan
6839,wooden,6,1,NaN,1.0,0.000001,NaN,0.000001,0,NaN,0.0,2.50,NaN,2.70,nan
6840,woodpecker,10,1,NaN,1.0,0.000001,NaN,0.000001,0,NaN,0.0,5.30,NaN,5.55,nan
6870,wounds,6,1,NaN,1.0,0.000001,NaN,0.000001,3,NaN,1.0,2.00,NaN,2.50,nan


In [ ]:
# 중요 결론,, HAL을 통해 POS가 달리지 않은 단어들은 모두 textbook에서만 나왔다!!!!
# -> 얘네들은 POS_tagging하자!! (nltk)
CSAT_minus_HAL.describe()

,Length,CSAT_Freq,CSAT_only_Freq,Textbook_only_Freq,CSAT_RFreq,CSAT_only_RFreq,Textbook_only_RFreq,Ortho_N_CSAT,Ortho_N_CSAT(only),Ortho_N_Textbook(only),OLD20_CSAT,OLD20_CSAT(only),OLD20_Textbook(only)
count,384.000000,384.000000,0.0,384.000000,3.840000e+02,0.0,3.840000e+02,384.000000,0.0,384.000000,384.000000,0.0,384.000000
mean,7.166667,1.114583,NaN,1.114583,1.114583e-06,NaN,1.114583e-06,0.731771,NaN,0.286458,2.937370,NaN,3.498438
std,2.331157,0.364761,NaN,0.364761,3.647609e-07,NaN,3.647609e-07,1.589269,NaN,0.846571,1.067104,NaN,1.269942
min,3.000000,1.000000,NaN,1.000000,1.000000e-06,NaN,1.000000e-06,0.000000,NaN,0.000000,1.300000,NaN,1.550000
25%,5.000000,1.000000,NaN,1.000000,1.000000e-06,NaN,1.000000e-06,0.000000,NaN,0.000000,2.200000,NaN,2.600000
50%,7.000000,1.000000,NaN,1.000000,1.000000e-06,NaN,1.000000e-06,0.000000,NaN,0.000000,2.650000,NaN,3.325000
75%,8.250000,1.000000,NaN,1.000000,1.000000e-06,NaN,1.000000e-06,1.000000,NaN,0.000000,3.550000,NaN,4.300000
max,16.000000,3.000000,NaN,3.000000,3.000000e-06,NaN,3.000000e-06,11.000000,NaN,6.000000,8.900000,NaN,9.500000


### HAL

In [169]:
# 모든 word에 대해 HAL_freq는 정의되어 있음.
E_lexicon_df[E_lexicon_df['Freq_HAL'] == None]

,Word,Length,Freq_HAL,Log_Freq_HAL,SUBTLWF,LgSUBTLWF,SUBTLCD,LgSUBTLCD,Ortho_N,OLD,POS


In [170]:
# HAL_df에는 SUBTLEX
HAL_df = E_lexicon_df.drop(columns=['SUBTLWF', 'LgSUBTLWF', 'SUBTLCD', 'LgSUBTLCD'])
HAL_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'HAL.xlsx'), index=False)
HAL_df

,Word,Length,Freq_HAL,Log_Freq_HAL,Ortho_N,OLD,POS
0,a,1,10610626,16.177,1,1.45,minor|NN
1,aah,3,222,5.403,2,1.85,None
2,aaron,5,10806,9.288,3,1.85,NN
3,aback,5,387,5.958,0,1.95,RB
4,abacus,6,513,6.240,0,2.90,NN
...,...,...,...,...,...,...,...
40476,zoom,4,4920,8.501,4,1.70,VB|NN
40477,zooming,7,523,6.260,3,1.85,VB
40478,zooms,5,385,5.953,3,1.80,VB
40479,zucchini,8,314,5.749,0,3.75,NN


In [171]:
HAL_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40481 entries, 0 to 40480
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Word          40481 non-null  object 
 1   Length        40481 non-null  int64  
 2   Freq_HAL      40481 non-null  int64  
 3   Log_Freq_HAL  40481 non-null  float64
 4   Ortho_N       40481 non-null  int64  
 5   OLD           39382 non-null  float64
 6   POS           39903 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 2.2+ MB


### SUBTLEX

In [ ]:
# SUBTLEX에는 없는 단어들 존재. 
E_lexicon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40481 entries, 0 to 40480
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Word          40481 non-null  object 
 1   Length        40481 non-null  int64  
 2   Freq_HAL      40481 non-null  int64  
 3   Log_Freq_HAL  40481 non-null  float64
 4   SUBTLWF       35515 non-null  float64
 5   LgSUBTLWF     35515 non-null  float64
 6   SUBTLCD       35515 non-null  object 
 7   LgSUBTLCD     35515 non-null  object 
 8   Ortho_N       40481 non-null  int64  
 9   OLD           39382 non-null  float64
 10  POS           39903 non-null  object 
dtypes: float64(4), int64(3), object(4)
memory usage: 3.4+ MB


In [166]:
SUBTLEX_df = E_lexicon_df.dropna(subset=['SUBTLWF']).reset_index(drop=True).drop(columns=['Freq_HAL', 'Log_Freq_HAL'])
SUBTLEX_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'SUBTLEX.xlsx'), index=False)
SUBTLEX_df

,Word,Length,SUBTLWF,LgSUBTLWF,SUBTLCD,LgSUBTLCD,Ortho_N,OLD,POS
0,a,1,20415.27,6.018,99.930,3.923,1,1.45,minor|NN
1,aah,3,52.71,3.430,7.560,2.803,2,1.85,None
2,aaron,5,14.65,2.874,1.930,2.212,3,1.85,NN
3,aback,5,0.29,1.204,0.180,1.204,0,1.95,RB
4,abacus,6,0.24,1.114,0.120,1.041,0,2.90,NN
...,...,...,...,...,...,...,...,...,...
35510,zoom,4,3.55,2.260,1.290,2.037,4,1.70,VB|NN
35511,zooming,7,0.63,1.518,0.310,1.431,3,1.85,VB
35512,zooms,5,0.06,0.602,0.040,0.602,3,1.80,VB
35513,zucchini,8,0.96,1.699,0.250,1.342,0,3.75,NN


In [172]:
SUBTLEX_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35515 entries, 0 to 35514
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Word       35515 non-null  object 
 1   Length     35515 non-null  int64  
 2   SUBTLWF    35515 non-null  float64
 3   LgSUBTLWF  35515 non-null  float64
 4   SUBTLCD    35515 non-null  object 
 5   LgSUBTLCD  35515 non-null  object 
 6   Ortho_N    35515 non-null  int64  
 7   OLD        35515 non-null  float64
 8   POS        35368 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 2.4+ MB


## CSAT + E-lexicon-proj

In [ ]:
CSAT_E_lexicon_df = pd.merge(CSAT_df.drop(columns=['POS', 'Length']), E_lexicon_df, how='outer', on='Word')
CSAT_E_lexicon_df = CSAT_E_lexicon_df[['Word', 'Length', 'CSAT_Freq', 'CSAT_only_Freq', 'Textbook_only_Freq', 'Freq_HAL', 'SUBTLWF', 'CSAT_RFreq', 'CSAT_only_RFreq', 'Textbook_only_RFreq', 'Ortho_N_CSAT', 'Ortho_N_CSAT(only)', 'Ortho_N_Textbook(only)', 'Ortho_N' ,'OLD20_CSAT','OLD20_CSAT(only)','OLD20_Textbook(only)','OLD', 'SUBTLCD', 'POS']]
CSAT_E_lexicon_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_E_lexicon.xlsx'), index=False)
CSAT_E_lexicon_df

,Word,Length,CSAT_Freq,CSAT_only_Freq,Textbook_only_Freq,Freq_HAL,SUBTLWF,CSAT_RFreq,CSAT_only_RFreq,Textbook_only_RFreq,Ortho_N_CSAT,Ortho_N_CSAT(only),Ortho_N_Textbook(only),Ortho_N,OLD20_CSAT,OLD20_CSAT(only),OLD20_Textbook(only),OLD,SUBTLCD,POS
0,a,1,1284.0,1098.0,186.0,10610626,20415.27,0.001284,0.001098,0.000186,1.0,1.0,1.0,1,1.5,1.5,1.65,1.45,99.930,minor|NN
1,aah,3,NaN,NaN,NaN,222,52.71,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,1.85,7.560,None
2,aaron,5,NaN,NaN,NaN,10806,14.65,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,1.85,1.930,NN
3,aback,5,NaN,NaN,NaN,387,0.29,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,1.95,0.180,RB
4,abacus,6,NaN,NaN,NaN,513,0.24,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,2.90,0.120,NN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40476,zoom,4,NaN,NaN,NaN,4920,3.55,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,1.70,1.290,VB|NN
40477,zooming,7,NaN,NaN,NaN,523,0.63,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,1.85,0.310,VB
40478,zooms,5,NaN,NaN,NaN,385,0.06,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,1.80,0.040,VB
40479,zucchini,8,NaN,NaN,NaN,314,0.96,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,3.75,0.250,NN


In [259]:
CSAT_E_lexicon_df.columns

Index(['Word', 'Length', 'CSAT_Freq', 'CSAT_only_Freq', 'Textbook_only_Freq',
       'Freq_HAL', 'SUBTLWF', 'CSAT_RFreq', 'CSAT_only_RFreq',
       'Textbook_only_RFreq', 'Ortho_N_CSAT', 'Ortho_N_CSAT(only)',
       'Ortho_N_Textbook(only)', 'Ortho_N', 'OLD20_CSAT', 'OLD20_CSAT(only)',
       'OLD20_Textbook(only)', 'OLD', 'SUBTLCD', 'POS'],
      dtype='object')

## CSAT + HAL

In [250]:
CSAT_HAL_df = pd.merge(CSAT_df.drop(columns=['POS', 'Length']), HAL_df, how='outer', on='Word')
CSAT_HAL_df = CSAT_HAL_df[['Word', 'Length', 'CSAT_Freq', 'CSAT_only_Freq', 'Textbook_only_Freq', 'Freq_HAL', 'CSAT_RFreq', 'CSAT_only_RFreq', 'Textbook_only_RFreq', 'Ortho_N_CSAT', 'Ortho_N_CSAT(only)', 'Ortho_N_Textbook(only)', 'Ortho_N' ,'OLD20_CSAT','OLD20_CSAT(only)','OLD20_Textbook(only)','OLD', 'POS']]
CSAT_HAL_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_HAL.xlsx'), index=False)
CSAT_HAL_df

,Word,Length,CSAT_Freq,CSAT_only_Freq,Textbook_only_Freq,Freq_HAL,CSAT_RFreq,CSAT_only_RFreq,Textbook_only_RFreq,Ortho_N_CSAT,Ortho_N_CSAT(only),Ortho_N_Textbook(only),Ortho_N,OLD20_CSAT,OLD20_CSAT(only),OLD20_Textbook(only),OLD,POS
0,a,1,1284.0,1098.0,186.0,10610626,0.001284,0.001098,0.000186,1.0,1.0,1.0,1,1.5,1.5,1.65,1.45,minor|NN
1,aah,3,NaN,NaN,NaN,222,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,1.85,None
2,aaron,5,NaN,NaN,NaN,10806,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,1.85,NN
3,aback,5,NaN,NaN,NaN,387,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,1.95,RB
4,abacus,6,NaN,NaN,NaN,513,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,2.90,NN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40476,zoom,4,NaN,NaN,NaN,4920,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,1.70,VB|NN
40477,zooming,7,NaN,NaN,NaN,523,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,1.85,VB
40478,zooms,5,NaN,NaN,NaN,385,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,1.80,VB
40479,zucchini,8,NaN,NaN,NaN,314,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,3.75,NN


## CSAT + SUBTLEX

In [ ]:
CSAT_SUBTLEX_df = pd.merge(CSAT_df.drop(columns=['POS', 'Length']), SUBTLEX_df, how='outer', on='Word')
CSAT_SUBTLEX_df = CSAT_SUBTLEX_df[['Word', 'Length', 'CSAT_Freq', 'CSAT_only_Freq', 'Textbook_only_Freq', 'SUBTLWF', 'CSAT_RFreq', 'CSAT_only_RFreq', 'Textbook_only_RFreq', 'Ortho_N_CSAT', 'Ortho_N_CSAT(only)', 'Ortho_N_Textbook(only)', 'Ortho_N' ,'OLD20_CSAT','OLD20_CSAT(only)','OLD20_Textbook(only)','OLD', 'SUBTLCD','POS']]
CSAT_SUBTLEX_df.to_excel(os.path.join(corpora_dir, 'merged_corpora', 'CSAT_SUBTLEX.xlsx'), index=False)
CSAT_SUBTLEX_df

,Word,Length,CSAT_Freq,CSAT_only_Freq,Textbook_only_Freq,SUBTLWF,CSAT_RFreq,CSAT_only_RFreq,Textbook_only_RFreq,Ortho_N_CSAT,Ortho_N_CSAT(only),Ortho_N_Textbook(only),Ortho_N,OLD20_CSAT,OLD20_CSAT(only),OLD20_Textbook(only),OLD,SUBTLCD,POS
0,a,1.0,1284.0,1098.0,186.0,20415.27,0.001284,0.001098,0.000186,1.0,1.0,1.0,1.0,1.5,1.5,1.65,1.45,99.930,minor|NN
1,aah,3.0,NaN,NaN,NaN,52.71,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.85,7.560,None
2,aaron,5.0,NaN,NaN,NaN,14.65,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.85,1.930,NN
3,aback,5.0,NaN,NaN,NaN,0.29,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.95,0.180,RB
4,abacus,6.0,NaN,NaN,NaN,0.24,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2.90,0.120,NN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35598,zoom,4.0,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,1.70,1.290,VB|NN
35599,zooming,7.0,NaN,NaN,NaN,0.63,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.85,0.310,VB
35600,zooms,5.0,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.80,0.040,VB
35601,zucchini,8.0,NaN,NaN,NaN,0.96,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,3.75,0.250,NN
